# Utilization Prediction: Prediction of hourly utilization of the two sites

## Import libraries

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
from keras import Sequential
from keras.layers import Dense

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## Load and prepare data

In [20]:
# Load data
charging_data = pd.read_pickle("data/charging_modified.pkl")
weather_data = pd.read_csv("data/weather_modified.csv")

# TODO: more filtering of features
# drop not important columns in weather data
weather_data.drop(columns = ["city", "cloud_cover_description", "pressure", "felt_temperature"], inplace = True)
weather_data["timestamp"] = pd.to_datetime(weather_data["timestamp"], utc = True).dt.floor(freq = "H")

# drop not important columns in charging data
charging_data.drop(columns = ["kWhPerMinute", "NoChargingTime", "user_kWhRequested", "user_modifiedAt", "doneChargingTime", "NoChargingTimeMinutes", "spaceID", "stationID", 
                              "user_userID", "ChargingTime", "user_paymentRequired_values"], inplace = True)

# Delete rows with null values
charging_data = charging_data.dropna()

# delete duplicate row index
charging_data.reset_index(drop = True, inplace = True)

charging_data = charging_data.astype({
    "kWhDelivered": "int64",
    "ChargingTimeMinutes": "int64",
    "user_minutesAvailable": "int64",
    "user_milesRequested": "int64",
    "user_WhPerMile": "int64",
    "siteID": "int64"
})

weather_data.astype({
    "temperature": "float",
    "cloud_cover": "float",
    "windspeed": "float",
    "precipitation": "float"
})

# check user_paymentRequired_values again

,timestamp,temperature,cloud_cover,windspeed,precipitation
0,2018-01-01 08:00:00+00:00,9.0,33.0,9.0,0.0
1,2018-01-01 09:00:00+00:00,9.0,33.0,0.0,0.0
2,2018-01-01 10:00:00+00:00,9.0,21.0,0.0,0.0
3,2018-01-01 11:00:00+00:00,9.0,29.0,0.0,0.0
4,2018-01-01 12:00:00+00:00,8.0,33.0,0.0,0.0
...,...,...,...,...,...
29239,2021-01-01 03:00:00+00:00,13.0,33.0,0.0,0.0
29240,2021-01-01 04:00:00+00:00,12.0,33.0,11.0,0.0
29241,2021-01-01 05:00:00+00:00,12.0,33.0,9.0,0.0
29242,2021-01-01 06:00:00+00:00,11.0,33.0,13.0,0.0


## Create feature vector X and labels Y

In [21]:
# split time interval between connectionTime and disconnectTime in minutes for every hour
def split_time_interval(index, start_date, end_date):
    indices = pd.DatetimeIndex([start_date])
    indices = indices.append(pd.date_range(start_date.ceil(freq = "H"), end_date.floor(freq = "H"), freq = "H"))
    indices = indices.append(pd.DatetimeIndex([end_date]))

    time_list = list(map(lambda x : 60 if x == 0 else x, indices.minute))
    time_list[0] = 60 - time_list[0]
    
    return list((time_list, indices))


# creates table with session id, charging time in minutes per hour and timestamps
def create_minute_table(id, minute_array, siteID):
    return list(zip([id] * len(minute_array[0]), minute_array[0], minute_array[1], [siteID] * len(minute_array[0])))

# groups entires by timestamp
def group_by_timestamp(X):
    X = X.groupby("timestamp").agg({"kWhDelivered": "sum",
                                     "ChargingTimeMinutes": "sum",
                                     "siteID": "mean",
                                     "month": "mean",
                                     "user_minutesAvailable": "sum",
                                     "user_milesRequested": "sum",
                                     "user_WhPerMile": "mean"
                               })
    return X

In [22]:
# concat single minute lists to one big list with lists of tuples containing id, charging times in minutes per hour and timestamps
temp = []

for index in charging_data.index :
    charging_times = split_time_interval(charging_data.index[index], 
                                         charging_data.loc[index, "connectionTime"], 
                                         charging_data.loc[index, "disconnectTime"])
    temp.append(create_minute_table(charging_data.loc[index, "id"], charging_times, charging_data.loc[index, "siteID"]))
    
charging_data.drop("siteID", inplace = True, axis = 1)

# parse list of lists of triples into list of three lists: ids, charging time in minutes per hour and timestamps
ids, charging_times, timestamps, siteIDs = map(list, zip(*[tuple for list in temp for tuple in list]))

# create data.frame with four columns: session id, charging time, starting timestamp in minutes for every hour and siteID
# TODO: id kinda useless
charging_time_per_hour = pd.DataFrame(
                            {"id": ids,
                             "charging_time_per_hour": charging_times,
                             "timestamp": timestamps,
                             "siteID": siteIDs})
charging_time_per_hour.loc[:, "timestamp"] = charging_time_per_hour.loc[:, "timestamp"].dt.floor(freq = "H")

### Labels Y:

In [23]:
# roughly 50 EV charging stations per site * 60 minutes
maximum_utilization = 50 * 60

# divide data set into two sets: one for each site
Y1 = charging_time_per_hour.loc[charging_time_per_hour["siteID"] == 1]
Y2 = charging_time_per_hour.loc[charging_time_per_hour["siteID"] == 2]

print(len(Y1))
print(len(Y2))

Y1 = pd.Series(map(lambda x : x / maximum_utilization, Y1.groupby("timestamp").sum().loc[:, "charging_time_per_hour"]))
Y2 = pd.Series(map(lambda x : x / maximum_utilization, Y2.groupby("timestamp").sum().loc[:, "charging_time_per_hour"]))

Y = pd.concat([Y1, Y2], axis = 0)
Y

281798
118826


0        0.021000
1        0.174667
2        0.300000
3        0.300000
4        0.300000
           ...   
16250    0.271000
16251    0.267000
16252    0.251333
16253    0.209667
16254    0.027333
Length: 31201, dtype: float64

### Feature Vector X:

In [43]:
# join charge time per hour with the rest of the charging data
X_temp = charging_time_per_hour.merge(charging_data, how = "left", on = "id").drop(["connectionTime", "disconnectTime", "user_requestedDeparture", "id", "userID", "charging_time_per_hour"], axis = 1)

# divide data set into two sets: one for each site
X1 = X_temp.loc[X_temp["siteID"] == 1]
X2 = X_temp.loc[X_temp["siteID"] == 2]

# group columns by timestamps
X1 = group_by_timestamp(X1)
X2 = group_by_timestamp(X2)

# combine both subsets together into one
X = pd.concat([X1, X2], axis = 0)
X = X.astype({"month": "int64",
              "siteID": "int64"})

# extract months from timestamp
X["month"] = list(map(lambda x: x.month, X.index))

# add weather data to feature vector
X = X.merge(weather_data, how = "left", on = "timestamp").drop_duplicates(["timestamp", "siteID"]).fillna(0).iloc[:, 1:3]
#X = X.drop("timestamp", axis = 1)
#X = pd.get_dummies(X, columns = ["siteID"])
X

,kWhDelivered,ChargingTimeMinutes
0,53,1860
1,247,8734
4,247,8734
5,247,8734
6,247,8734
...,...,...
34107,258,3166
34108,303,3582
34109,337,4023
34110,158,2271


## Cross-validation: split data in training and test set

In [44]:
# Split data into training set and testing set nad normalize
scaler = StandardScaler()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.3, random_state = 30)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
type(Y_train)

pandas.core.series.Series

## Prediction model 1: Neural Network

In [45]:
# Build the neural network
network = Sequential()

# Add layers
network.add(Dense(input_shape = (2,), units = 15, activation = "relu"))
network.add(Dense(units = 15, activation = "relu"))
network.add(Dense(units = 1, activation = "sigmoid"))

In [46]:
# Compiling the neural network
network.compile(optimizer = "adam", 
                   loss = "mean_squared_error", 
                   metrics = ["mean_squared_error"])

In [47]:
network.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ dense_9 (Dense)                    │ (None, 15)                    │          45 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_10 (Dense)                   │ (None, 15)                    │         240 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_11 (Dense)                   │ (None, 1)                     │          16 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 301 (1.18 KB)

 Trainable params: 301 (1.18 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# Fitting the Neural Network
network.fit(X_train, Y_train, batch_size=50, epochs=100)

Epoch 1/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0787 - mean_squared_error: 0.0787
Epoch 2/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0120 - mean_squared_error: 0.0120
Epoch 3/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0114 - mean_squared_error: 0.0114
Epoch 4/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0108 - mean_squared_error: 0.0108
Epoch 5/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0114 - mean_squared_error: 0.0114
Epoch 6/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0115 - mean_squared_error: 0.0115
Epoch 7/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0120 - mean_squared_error: 0.0120
Epoch 8/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0114 - mean_squared_error: 0.0114
Epoch 9/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0111 - mean_squared_error: 0.0111
Epoch 10/100
437/437 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0110 - mean_squared_error: 0.0110
Epoch 11/100
437/437 ━━━━━━━━

In [49]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

mean_squared_error = mean_squared_error(Y_test, network.predict(X_test))
mean_absolute_error = mean_absolute_error(Y_test, network.predict(X_test))
mean_squared_log_error = mean_squared_log_error(Y_test, network.predict(X_test))
print(mean_squared_error)
print(mean_absolute_error)
print(mean_squared_log_error)

293/293 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
293/293 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
293/293 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
0.010701849144650406
0.04534814496281115
0.003142158002753584
